### Import libiraries.

In [1]:
from bs4 import BeautifulSoup
import requests 
import time
import re

### Function to check if the link in parentheses, cite note, help link or not valid. 

In [2]:
def check_par(paragraph, link_text, link_url):
    # Get all text in parentheses in the  given paragraph using regular expression.
    word_list1 = re.findall('\(([^)]+)', paragraph)
    word_list2 = re.findall('\[([^]]+)', paragraph)
    
    # If the link doesn't exist, return true
    if(link_url == None) : return True
    # If the link is cite note, return true
    if('#cite' in link_url): return True
    # If the link is help:IPA link, return true
    if('Help:IPA' in link_url): return True
    # If the link is in parentheses, return true
    for subtext in word_list1:
        if(link_text in subtext): return True
    for subtext in word_list2:
        if(link_text in subtext): return True
    # Otherwise, return false    
    return False  

### Function to find the first normal link in paragraphs.

In [3]:
def find_link(paragraphs , loop_option):
    
    
        link_url = ''
        # For each paragraph in given paragraphs:
        for paragraph in paragraphs:
            # If there is links in the paragraph:
            if(paragraph.find("a")):
                para_text = paragraph.getText()
                links = paragraph.find_all('a')
                # For each link in this paragraph:
                for link in links:
                    link_url = link.get('href')
                    link_text = link.text
                    
                    # Check for this link using check_par function
                    check = check_par(para_text, link_text, link_url)
                                      
                    if(loop_option == 'yes'): # If choose to stop when stuck in loop:             
                        # If stuck in loop, stop.
                        if(link_url in history): return 'We stuck in a loop :('
                        # If the link is normal return. Otherwise, find the next link.              
                        if(check == False): return link_url
                    
                    else: # If choose to complete with second normal link when stuck in loop:
                        # Return the link only if it's normal and not repeated.              
                        if((check == False) and (link_url not in history)):
                            return link_url

In [4]:
URL = input("Please Enter the URL of the wikipedia page ? ")
loop_option = input('Do you want to stop when stuck in loop or complete with second normal link?\ntype yes for first option. ')
page = requests.get(URL).text
history = []

while True:
    # Get the HTML of the page
    soup = BeautifulSoup(page , 'html.parser')
    # Find the class of the main page article
    divv = soup.find("div", {"id": "mw-content-text"})
    div = divv.find("div" , {"class": "mw-parser-output"})
    # Get all direct paragraph children  
    p = div.find_all("p", attrs={'class': None} , recursive=False)
    
    # Find first normal link using find_link function and print it.
    link = find_link(p, loop_option.lower())
    print(link) 
        
    # If we have repeated link with stop option, stop    
    if(link ==  'We stuck in a loop :('): break    
        
    # If there are no links in all paragraphs, then that's a dead-end page    
    if(link == ""):
        print("This page has no links (Dead-End page)")
        break
        
    # Handle the link was to another site ( like wiktionary )      
    if("https" in link):
        print('We are outside wikipeidia website')
        page = requests.get(link).text
        time.sleep(0.5)
        continue
        

        
    history.append(link)    
    # Stop If the link is Philosophy page. Otherwise, Continue
    if(link != "/wiki/Philosophy"):
        url = "https://en.wikipedia.org" + link
        page = requests.get(url).text
        time.sleep(0.5)
    else:
        print("You have arrived :)")
        break

Please Enter the URL of the wikipedia page ? https://en.wikipedia.org/wiki/Special:Random
Do you want to stop when stuck in loop or complete with second normal link?
type yes for first option. no
/wiki/Hindi
/wiki/Indo-Aryan_language
/wiki/South_Asia
/wiki/Asia
/wiki/Earth
/wiki/Planet
/wiki/Astronomical_body
/wiki/Astronomy
/wiki/Natural_science
/wiki/Branches_of_science
/wiki/Science
/wiki/Testability
/wiki/Empirical
/wiki/Information
/wiki/Uncertainty
/wiki/Epistemic
/wiki/Greek_language
/wiki/Indo-European_languages
/wiki/Language_family
/wiki/Language
/wiki/Grammar
/wiki/Linguistics
/wiki/Theoretical_linguistics
/wiki/Theory_of_language
/wiki/Philosophy_of_language#Nature_of_language
/wiki/Analytic_philosophy
/wiki/Philosophy
You have arrived :)


# Notes:

### Science and Language pages lead to loop.